### Sum() reducer using cuda-kernels (very simplified)

(just for comparison)

In [1]:
import cupy as cp

cuda_kernel = """
extern "C" {
    __global__ void awkward_reduce_sum_a(int* toptr, int* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;

       if (thread_id < outlength) {
          toptr[thread_id] = 0;
       }
    }
}
extern "C" {
    __global__ void awkward_reduce_sum_b(int* toptr, int* fromptr, int* parents, int lenparents, int outlength) {
       int thread_id = blockIdx.x * blockDim.x + threadIdx.x;
       int stride = blockDim.x * gridDim.x;

       for (int i = thread_id; i < lenparents; i += stride) {
           atomicAdd(&toptr[parents[i]], fromptr[i]);
       }
    }
}
"""

In [3]:
parents = cp.array([0, 1, 1, 2, 2, 3, 3, 3, 5], dtype=cp.int32)
fromptr = cp.array([1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=cp.int32)
lenparents = len(parents)
outlength = int(cp.max(parents)) + 1
toptr = cp.zeros(outlength, dtype=cp.int32)

block_size = 256
grid_size = (lenparents + block_size - 1) // block_size

raw_module = cp.RawModule(code=cuda_kernel)

awkward_reduce_sum_a = raw_module.get_function('awkward_reduce_sum_a')
awkward_reduce_sum_b = raw_module.get_function('awkward_reduce_sum_b')

awkward_reduce_sum_a((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))
awkward_reduce_sum_b((grid_size,), (block_size,), (toptr, fromptr, parents, lenparents, outlength))

toptr

array([ 1,  5,  9, 21,  0,  9], dtype=int32)

### Sum() reducer using CCCL

In [2]:
# An attempt to recreate studies/cuda-kernels/reducers/awkward_reduce_sum_atomics.py using cccl instead of raw cuda kernels:
import awkward as ak
import cupy as cp
import numpy as np

from cuda.compute import segmented_reduce

def cccl_sum(input_data, offsets):

    def sum_op(a, b):
        return a+b

    # Prepare the start and end offsets
    start_o = offsets[:-1]
    end_o = offsets[1:]

    # Prepare the output array
    n_segments = start_o.size
    output = cp.empty(n_segments, dtype=cp.int32)

    # Initial value for the reduction
    h_init = np.array([0], dtype=np.int32)

    # Perform the segmented reduce
    segmented_reduce(
        input_data, output, start_o, end_o, sum_op, h_init, n_segments
    )

    return output.get()

In [4]:
awkward_array = ak.Array([[1], [2, 3], [4, 5], [6, 7, 8], [], [9]], backend = 'cuda')
input_data = awkward_array.layout.content.data
offsets = awkward_array.layout.offsets.data

output = cccl_sum(input_data, offsets)

print(f"Segmented reduce result: {output}")

Segmented reduce result: [ 1  5  9 21  0  9]


### Argmax() reducer using CCCL

In [8]:
import awkward as ak
import cupy as cp
import numpy as np
import time
import nvtx

from cuda.compute import segmented_reduce, ZipIterator, gpu_struct, reduce_into

# An attempt to recreate studies/cuda-kernels/reducers/awkward_reduce_argmax_tree_reduction.py using cccl instead of raw cuda kernels
def cccl_argmax(awkward_array):
    @gpu_struct
    class ak_array:
        data: cp.float64
        local_index: cp.int64

    # compare the values of the arrays
    def max_op(a: ak_array, b: ak_array):
        return a if a.data > b.data else b

    input_data = awkward_array.layout.content.data
    
    # use an internal awkward function to get the local indicies
    local_indicies = ak.local_index(awkward_array, axis=1)
    local_indicies = local_indicies.layout.content.data

    #Combine data and their indicies into a single structure
    input_struct = ZipIterator(input_data, local_indicies)

    # Prepare the start and end offsets
    offsets = awkward_array.layout.offsets.data
    start_o = offsets[:-1]
    end_o = offsets[1:]

    # Prepare the output array
    n_segments = start_o.size
    output = cp.zeros([n_segments], dtype= ak_array.dtype)

    # Initial value for the reduction
    h_init = ak_array(-1, -1)

    # Perform the segmented reduce
    segmented_reduce(
        input_struct, output, start_o, end_o, max_op, h_init, n_segments
    )

    return output.view(cp.int64).reshape(-1, 2)[:, 1]

In [9]:
awkward_array = ak.Array([[1], [2, 3], [4, 5], [6, 7, 8], [], [9]], backend = 'cuda')

cccl_argmax(awkward_array)

array([ 0,  1,  1,  2, -1,  0])